## Import

In [3]:
from tqdm import tnrange
from tqdm import tqdm
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

import time
import pandas as pd
import numpy as np
import re
import requests

## Function

In [52]:
def cleansing_text(x) :
    output = x[0].text
    output = re.sub("[\n\t]","",output)
    return output

## Crawling

* 투자 전략 크롤링

In [39]:
csv_save_path = "/home/ubunt/4-2/stock price/data/invest_csv/"

#### invest_list

In [8]:
base_url = "https://finance.naver.com/research/"
start_index = 1

In [37]:
for index in tqdm_notebook(range(start_index,594)) :
    url = "https://finance.naver.com/research/invest_list.nhn?&page={}".format(index)
    res = requests.get(url)
    assert res.status_code == 200
    
    soup = BeautifulSoup(res.text, "html.parser")
    tmp_set = soup.select("table.type_1 td a")
    page_titles = [x.text for x in tmp_set if len(x.text) > 1]
    page_urls = [base_url + x.get("href") for x in tmp_set if "https://" not in x.get("href") and "http://" not in x.get("href")]
    
    tmp_date = soup.select("table.type_1 td.date")
    page_dates = [x.text for x in tmp_date if "." in x.text]
    page_views = [x.text for x in tmp_date if "." not in x.text]
    
    page_df = pd.DataFrame({"date" : page_dates, "title" : page_titles, "view" : page_views, "url" : page_urls})
    
    if start_index == 1 :
        invest_inform = page_df
    else :
        invest_inform = invest_inform.append(page_df)
    start_index += 1

In [38]:
invest_inform.index = [x for x in range(invest_inform.shape[0])]

In [40]:
invest_inform.to_csv(csv_save_path + "invest_inform_2019092723.csv", index = False)

#### invest text

In [87]:
start_index = 0

In [88]:
initial_idx = start_index
for idx,url in enumerate(tqdm_notebook(invest_inform.url[start_index:])) :
    whole_idx = initial_idx + idx
    if int(invest_inform.date[whole_idx][:2]) >= 10 :
        res = requests.get(url)
        assert res.status_code == 200
        soup = BeautifulSoup(res.text, "html.parser")
        tmp_text = soup.select("td.view_cnt div")
        texts = cleansing_text(tmp_text)

        text_df = pd.DataFrame({"text" : [texts], "url" : [url]})

        if start_index == 0 :
            invest_text_df = text_df
        else :
            invest_text_df = invest_text_df.append(text_df)
        start_index += 1
    else :
        break

In [89]:
invest_text_df.index = [x for x in range(invest_text_df.shape[0])]

In [91]:
invest_text_df.to_csv(csv_save_path + "invest_text_df_2019092800.csv", index= False)